# <span style="color: rgb(138, 92, 245);">I. Encuesta en Aerolíneas</span>

## <span style="color: rgb(138, 92, 245);">1. Introducción</span>

Comenzamos por importar los datos referentes a este ejercicio. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
df_1 = pd.read_csv('data/data_practica_encuesta.csv')

In [5]:
df_1.head()

,Id,Servicio wifi a bordo,Hora de salida/llegada conveniente,Facilidad de reserva en línea,Ubicación de la puerta,Alimentos y bebidas,Embarque en línea,Comodidad del asiento,Entretenimiento a bordo,Servicio a bordo,Servicio de sala de piernas,Manejo de equipaje,Servicio de facturación,Limpieza
0,19556,5,4,3,4,3,4,3,5,5,5,5,2,5
1,90035,1,1,3,1,5,4,5,4,4,4,4,3,5
2,12360,2,0,2,4,2,2,2,2,4,1,3,2,2
3,77959,0,0,0,2,3,4,4,1,1,1,1,3,4
4,36875,2,3,4,3,4,1,2,2,2,2,2,4,4


El presente ejercicio tiene como objetivo analizar la opinión de los usuarios de una aerolínea sobre diversos aspectos del servicio. A partir de los datos proporcionados, se evaluará la calidad de los datos, se realizará un análisis exploratorio y se aplicará un Análisis Factorial para reducir la dimensionalidad y obtener componentes latentes que representen los factores subyacentes en la percepción de los usuarios.

## <span style="color: rgb(138, 92, 245);">2. Calidad de los datos</span>

**Objetivo**: validar si los datos son consistentes, completos y adecuados para el análisis.
Incluye:

* Verificación de valores nulos o atípicos.

* Revisión de tipos de datos.

* Identificación de valores extremos o inconsistentes.

 * Decisiones de limpieza o imputación.

In [ ]:
# Funcion Para Imprimir Porcetanje de Valores Nulos y Tipos de Datos
def print_data_info(df):
    print("Información del DataFrame:")
    print(df.info())
    print("\n Porcentaje de Valores nulos por columna:")
    print(df.isnull().mean() * 100)
    print("\n Cantidad de Valores Únicos por columna:")
    print(df.nunique())

In [14]:
# Imprimir Información del DataFrame y Porcentaje de Valores Nulos
print_data_info(df_1)

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25976 entries, 0 to 25975
Data columns (total 14 columns):
 #   Column                              Non-Null Count  Dtype
---  ------                              --------------  -----
 0   Id                                  25976 non-null  int64
 1   Servicio wifi a bordo               25976 non-null  int64
 2   Hora de salida/llegada conveniente  25976 non-null  int64
 3   Facilidad de reserva en línea       25976 non-null  int64
 4   Ubicación de la puerta              25976 non-null  int64
 5   Alimentos y bebidas                 25976 non-null  int64
 6   Embarque en línea                   25976 non-null  int64
 7   Comodidad del asiento               25976 non-null  int64
 8   Entretenimiento a bordo             25976 non-null  int64
 9   Servicio a bordo                    25976 non-null  int64
 10  Servicio de sala de piernas         25976 non-null  int64
 11  Manejo de equipaje                  2597

Los resultados anteriores nos indican que toda variable es numérica por lo que se consideraron aptas para el Análisis Factorial. Además, ninguna variable contiene valores nulos, por lo cuál no es necesario imputar o eliminar ninguna variable. Tampoco se tienen variables ***dummy***, pues ninguna es de tipo binaria (tampoco unaria).

# <span style="color: rgb(138, 92, 245);">2. FNA de Masa Mamaria</span>

In [6]:
df_2 = pd.read_csv('data/data_practica_cancer.csv')

# <span style="color: rgb(138, 92, 245);">3. Lenguage de Señas</span>

In [7]:
df_3 = pd.read_csv('data/data_practica_lenguaje_señas.csv')